>>>[5.1 Trivial Implementation](#folderId=1NMmMD5RX97aCsXDtRdiTFytxzVFeGSnW&updateTitle=true&scrollTo=-TLcDZv9QLDV)
>>>[5.3 Analyzing the Algorithm](#folderId=1NMmMD5RX97aCsXDtRdiTFytxzVFeGSnW&updateTitle=true&scrollTo=jnDBhClMVdlo)
>>>>[How is the length of the inout message affecting the speed of encrypting and decrypting?](#folderId=1NMmMD5RX97aCsXDtRdiTFytxzVFeGSnW&updateTitle=true&scrollTo=bNram5eGVnEZ)


In [1]:
import random
import math
import requests
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%run "source/RSA Helper Functions.ipynb"

To better understand the practical application of the theoretical principles we've discussed, let's walk through the key generation, encryption, and decryption processes involved in the RSA algorithm. For the sake of clarity, we'll use small numbers as the previous math example.

### 5.1 Trivial Implementation

First let's prepare the functions that will be needed to perform the 5 steps of the key generation process.

#### Generate prime numbers

In [2]:
def is_prime (number):
    if number < 2:
        return False
    for i in range (2, number // 2 + 1):
        if number % i == 0:
            return False
    return True

def get_random_number(n_bits):
    return random.randrange(2**(n_bits-1) + 1, 2**n_bits - 1)

def generate_prime(n_bits):
    while True:
        prime = get_random_number(n_bits)
        if is_prime(prime):
            return prime

#### Generate $\varphi(n)$

In [3]:
def eulers_totient(p, q):
    return (p-1) * (q-1)

#### Generate public exponent $e$

In [4]:
def generate_public_exponent(phi_n):
    e = random.randint (3, phi_n-1)
    while math.gcd(e, phi_n) != 1:
        e = random.randint (3, phi_n - 1)

    return e

#### Calculate the modular inverse $d$

In [5]:
def mod_inverse(e, phi):
    for d in range (3, phi):
        if (d * e) % phi == 1:
            return d
    raise ValueError ("Mod_inverse does not exist!")

#### Generate keys

Now let's combine the above functions to generate the public and private keys. The function `generate_keys()` will also return $n$, which will be used in subsequent encryption/decryption.

In [6]:
def generate_keys(p = None, q = None, e = None, n_bits = 8):
    # Step 1: Generate primes
    if ((not p) or (not q)):
        p, q = generate_prime(n_bits), generate_prime (n_bits)
        if p == q:
            q = generate_prime (n_bits)
    else:
        if (not is_prime(p) or not is_prime(q)):
            pass

    # Step 2: Generate the product of the primes
    n = p * q

    # Step 3: Generate φ(n)
    phi_n = eulers_totient(p, q)

    # Step 4: Generate the public exponent e
    if (not e):
        e = generate_public_exponent(phi_n)

    # Step 5: Calculate the modular inverse d
    d = mod_inverse(e, phi_n)

    return e, d, n

#### Encrypt/Decrypt the message

In [7]:
def encrypt_message_to_cipher(M, e, n):
    M_ascii_encoded = [ord(ch) for ch in M]
    cipher = [pow(ch, e, n) for ch in M_ascii_encoded]

    return cipher

def decrypt_cipher_to_message(C, d, n):
    C_ascii_encoded = [pow(ch, d, n) for ch in C]
    M = "".join (chr(ch) for ch in C_ascii_encoded)

    return M

#### Combining all components

In [8]:
e, d, n = generate_keys()

message = "A"
encrypted = encrypt_message_to_cipher(message, e, n)
decrypted = decrypt_cipher_to_message(encrypted, d, n)

print ("Public Key:  ", e)
print ("Private Key: ", d)
print ("n:           ", n)
print ("Original message:  ", message)
print ("Encrypted by RSA   ", encrypted)
print ("Decrypted message: ", decrypted)

Public Key:   19507
Private Key:  14443
n:            34571
Original message:   A
Encrypted by RSA    [2396]
Decrypted message:  A


#### Unit tests

Now is a good time to write some tests so that we know we haven't broken our implementation with future changes of the code.

In [9]:
def test_decrypting(message):
    e, d, n = generate_keys()
    encrypted = encrypt_message_to_cipher(message, e, n)
    decrypted = decrypt_cipher_to_message(encrypted, d, n)
    assert decrypted == message, "The decrypted message is not the same as the original message!"

def all_unit_tests(messages):
    for message in messages:
        test_decrypting(message)

    print("\033[32m ✅ All tests passed \033[0m")

In [10]:
file_6000_chars = Path("test_data/file_6000_chars.txt").read_text()

all_unit_tests(["A", "AB", "Hello and welcome to Sofia!", file_6000_chars])

 ✅ All tests passed 


### 5.2 Analyzing the Algorithm

In this section, we will explore the impact of modifying various parameters involved in the RSA algorithm, particularly focusing on how these changes affect 🔴 security and performance.

#### 5.2.1 How the size of $p$ and $q$ affects the time for their generation?

##### Explore the time to generate prime numbers of varying lengths

Let's explore the time required to generate prime numbers of varying lengths. We will generate specific primes by constraining the range to the desired prime. The current algorithm in function `generate_prime()` we created above (🔴link to section ) employs a brute-force approach, generating a random number within a specified range and then checking its primality using a brute-force method.

<div class="alert alert-block alert-warning" style="font-size:1.2em">
⚠️The below cell will run for $≈$3 min. If you want to run it set the flag to True. Otherwise you can inspect the output in the screenshot below the cell.
</div>

In [23]:
# function 'mean_time_msec' is defined in the companion notebook RSA Helper Functions.ipynb
def test_prime_generation_performance(n_bits_list, fn_generate_prime):
    return  [mean_time_msec(5)(fn_generate_prime)(n_bits) for n_bits in n_bits_list]

In [27]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

n_bits_list = [5, 10, 20, 25, 27, 30]
if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    print(test_prime_generation_performance(n_bits_list, generate_prime))

Output of the above cell:

![generate_primes_brute_force.png](images/generate_primes_brute_force.png)

Generating a 30 bit prime number using the brutre-force method took $≈$ 33 seconds. Clearly a performance hog! Let's try to improve the performance by using another algorithm like the Sieve of Eratosthenes.

##### Try to improve prime generation by using the Sieve of Eratosthenes

In [ ]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

# functions 'generate_prime_sieve_eratosthenes' and 'mean_time_msec' are defined in the companion notebook RSA Helper Functions.ipynb
if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [5, 10, 20, 25, 26, 27, 28, 29, 30]
    times_to_generate_prime_sieve = [mean_time_msec(3)(generate_prime_sieve_eratosthenes)(n_bits) for n_bits in n_bits_list]
    print(times_to_generate_prime_sieve)


print(test_prime_generation_performance(n_bits_list, generate_prime_sieve_eratosthenes))

Output of the above cell:

![generate_primes_sieve_er.png](images/generate_primes_sieve_er.png)

Generating a 30-bit prime number using the Sieve of Eratosthenes took 221 seconds. This is much worse performance. However, this is expected because the Sieve of Eratosthenes is designed to generate a list of all prime numbers up to a given limit, whereas in this case, we only needed one prime number within that range. As a result, the algorithm performed more work than was necessary to achieve the desired outcome, leading to the longer execution time.

##### Another improvement attempt by using the Miller-Rabin Primality test

Let's make another attempt to improve the prime generation with the Miller-Rabin primality test instead of the brute force `is_prime()`. This algorithm can produce false positves that is why it is executed multiple times to increase the probability of a correct result.

In [13]:
# function 'is_prime_miller_rabin' is defined in the companion notebook RSA Helper Functions.ipynb
def generate_prime_miller_rabin(n_bits):
    while True:
        prime = get_random_number(n_bits)
        if is_prime_miller_rabin(prime, k=20):
            return prime

In [14]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False

if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [256, 516, 1024, 2048, 4096]
    times_to_generate_prime_mr = [mean_time_msec(3)(generate_prime_miller_rabin)(n_bits) for n_bits in n_bits_list]
    print(times_to_generate_prime)

Output of the above cell:

![generate_primes_miller_rabin.png](images/generate_primes_miller_rabin.png)

We see that even though we saw good times for generation of small and mid-sized primes, even the Miller-Rabin algorithm fails in terms of performance for 2048-bit and 4096-bit primes. Let's make one more attempt at improving performance by combining the Sieve or Eratosthenes and the Rabin-Miller primality test. The idea is to generate less prime candidates by first excluding the non-primes using the sieve.

##### Yet another attempt to improve performance by combining the Seive of Eratosthenes and the Rabin-Miller primality test

In [15]:
def generate_prime_miller_rabin_sieve_er (n_bits):
    while True:
        prime_candidate = get_random_number_sieved(n_bits)
        if not is_prime_miller_rabin(prime_candidate):
            continue
        else:
            return prime_candidate

In [16]:
I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME = False
# function 'is_prime_miller_rabin_sieve_er' is defined in the companion notebook RSA Helper Functions.ipynb

if I_UNDERSTAND_THE_CELL_WILL_RUN_LONG_TIME:
    n_bits_list = [256, 516, 1024, 2048, 4096]
    times_to_generate_prime_mr_sieve = [mean_time_msec(5)(generate_prime_miller_rabin_sieve_er)(n_bits) for n_bits in n_bits_list]
    print(times_to_generate_prime_mr_sieve)

Output of the above cell:

![generate_primes_rabin_miller_sieve_er.png](images/generate_primes_rabin_miller_sieve_er.png)

Finally, we have enough information to compare the time for generating prime numbers of different lengths using two algorithms:

In [17]:
n_bits_list = [256, 516, 1024, 2048, 4096]
times_to_generate_prime_mr = [14.430100098252296, 58.95279999822378, 610.2232998237014, 16563.933900091797, 408703.5202998668]
times_to_generate_prime_mr_sieve = [17.45451996102929, 61.99582004919648, 398.5925999470055, 4936.42343999818, 27441.254119947553]

plt.figure(figsize=(10, 6))
plt.plot(n_bits_list, times_to_generate_prime_mr, marker='o', label='Rabin-Miller')
plt.plot(n_bits_list, times_to_generate_prime_mr_sieve, marker='o', label='Sieve of Eratosthenes and Rabin-Miller')
plt.xlabel('Prime Number Length (bits)')
plt.ylabel('Time (ms)')
plt.title('Generation of Prime Numbers of Different Length Using Two Algorithms')
plt.xticks(n_bits_list)
plt.legend()
plt.grid(True)
plt.show()

<div class="alert alert-danger" role="alert">
ДО ТУК!
</div>

Test the time it takes to encrypt and decrypt messages of different lengths and plot the result:

In [18]:

# OLD: #### How is the length of the input message affecting the speed of encrypting and decoding?
message_lengths = [1000, 50_000, 100_000, 150_000, 200_000, 250_000, 300_000, 400_000, 500_000]

long_text=""
if IS_GOOGLE_COLAB == True: # 🔴
    response = requests.get("https://raw.githubusercontent.com/MirkaIvanova/public_data/main/a_tale_of_two_cities_780000_chars.txt")
    long_text = response.text
else:
    raise FileNotFoundError("FIXME!")

long_text[:5]

e, d, n = generate_keys()

input_messages = [long_text[:n] for n in message_lengths]
encrypted_messages = [encrypt_message_to_cipher(message, e, n) for message in input_messages]
decrypted_messages = [decrypt_cipher_to_message(encrypted_message, d, n) for encrypted_message in encrypted_messages]

times_to_encrypt = [mean_time_msec(10)(encrypt_message_to_cipher)(message, e, n) for message in input_messages]
times_to_decrypt = [mean_time_msec(10)(decrypt_cipher_to_message)(encrypted_message, d, n) for encrypted_message in encrypted_messages]

print(times_to_encrypt)
print(times_to_decrypt)

plt.figure(figsize=(10, 6))
plt.plot(message_lengths, times_to_encrypt, marker='o', label='Encryption Time')
plt.plot(message_lengths, times_to_decrypt, marker='o', label='Decryption Time')

# Set the x-axis ticks to be proportionally spaced based on message_lengths
plt.xticks(message_lengths)

plt.xlabel('Message Length (characters)')
plt.ylabel('Time (ms)')
plt.title('Encryption and Decryption Times for Different Message Lengths')
plt.xticks(message_lengths)
plt.legend()
plt.grid(True)
plt.show()





NameError: name 'IS_GOOGLE_COLAB' is not defined